# Problema de la mochila 0/1 con Branch & Bound

En este notebook resolvemos el problema de la mochila 0/1 utilizando el enfoque de Branch & Bound.

## Planteo del problema

Dado un conjunto de $n$ objetos, cada uno con un peso $w_i$ y un valor $v_i$, y una capacidad máxima $C$ para la mochila, queremos seleccionar un subconjunto de objetos para maximizar el valor total sin exceder la capacidad.

## Formulación como problema de optimización

$$
\begin{align}
\text{Maximizar} \quad & \sum_{i=1}^{n} v_i\, x_i \\
\text{s.a.} \quad & \sum_{i=1}^{n} w_i\, x_i \le C \\
 & x_i \in \{0,1\} \quad \forall i \in \{1,\dots,n\}
\end{align}
$$

Donde $x_i = 1$ si el objeto $i$ se incluye, y $x_i = 0$ en caso contrario.

## Enfoque Branch & Bound
- El estado parcial se representa como un vector binario `x` de longitud $k \le n$ indicando decisiones para los primeros $k$ ítems.
- Se generan dos hijos por nodo: incluir o no incluir el siguiente objeto.
- Se calcula una cota superior (bound) del mejor valor posible desde ese estado. Si la cota es menor o igual que el mejor valor factible encontrado, se poda la rama.
- Usamos distintas funciones de cota para comparar su "tightness" (ajuste) mediante métricas de poda.


In [1]:
# Datos de entrada (puedes modificarlos para tus pruebas)

# Juego de datos 1: chico
weights = [2, 3, 4, 5]
values  = [3, 4, 5, 6]
capacity = 5

# Juego de datos 2: mediano (opcional; déjalo vacío o reemplázalo por otro set)
# Ejemplo comentado:
# weights = [4, 2, 3, 5, 1, 6, 7, 4]
# values  = [8, 4, 5, 10, 3, 14, 15, 8]
# capacity = 15

n = len(weights)


In [2]:
# Núcleo del Branch & Bound (plantilla para práctico)
from typing import List, Tuple
from collections import deque
import heapq

# --- Utilidades (dejadas listas) ---
def total_weight(x: List[int]) -> int:
    return sum(weights[i] for i, b in enumerate(x) if b == 1)

def total_value(x: List[int]) -> int:
    return sum(values[i] for i, b in enumerate(x) if b == 1)

def is_feasible(x: List[int]) -> bool:
    return total_weight(x) <= capacity

def generate_children(x: List[int]) -> List[List[int]]:
    i = len(x)
    if i >= n:
        return []
    return [x + [1], x + [0]]

optimal = float("-inf")
best_solution = None
nodes_generated = nodes_pruned = nodes_expanded = 0

def reset_stats():
    global optimal, best_solution, nodes_generated, nodes_pruned, nodes_expanded
    optimal = float("-inf")
    best_solution = None
    nodes_generated = 0
    nodes_pruned = 0
    nodes_expanded = 0

def branch_and_bound(x: List[int], bound_func) -> None:
    """Pseudocódigo sugerido:
      1) Poda por capacidad
      2) Cálculo de cota ub y poda por cota
      3) Si hoja: actualizar óptimo
      4) Si interno: generar hijos, contar nodes_generated y recursión
    """

    if is_feasible(x) == False:
        global nodes_pruned
        nodes_pruned += 1
        return
    
    ub = bound_func(x)
    global optimal
    if ub <= optimal:
        nodes_pruned += 1
        return
    
    if len(x) == n:
        v = total_value(x)
        if v > optimal:
            optimal = v
            global best_solution
            best_solution = x
        return
    
    children = generate_children(x)
    global nodes_generated, nodes_expanded
    nodes_generated += len(children)
    nodes_expanded += 1
    for child in children:
        branch_and_bound(child, bound_func)
    


In [3]:
# Cotas (bounds) — plantilla de práctico
from typing import List

class BoundStrategies:
    @staticmethod
    def _current_weight_and_value(x: List[int]):
        tw = tv = 0
        for i, take in enumerate(x):
            if take:
                tw += weights[i]
                tv += values[i]
        return tw, tv

    #COTA FRACCIONAL (RELAJACIÓN LINEAL)
    @staticmethod
    def fractional(x: List[int]) -> float:
        if not is_feasible(x):
            return 0
    
        peso_actual = total_weight(x)
        valor_actual = total_value(x)
        cap = capacity - peso_actual
    
        L = []
        for i in range(len(x), n):
            if weights[i] > 0:
                ratio = values[i] / weights[i]
                L.append((ratio, weights[i], values[i]))

        L.sort(key=lambda item: item[0], reverse=True)
        ub = valor_actual
        for ratio, wi, vi in L:
            if cap == 0:
                break
            if wi <= cap:
                ub += vi
                cap -= wi
            else:
                ub += ratio * cap
                cap = 0
                break

        return ub

    #COTA CON ÍTEMS COMPLETOS ORDENADOS POR V/W
    @staticmethod
    def whole_items_ratio(x: List[int]) -> float:
        """TODO: Implementar cota con ítems completos ordenados por v/w."""
        if not is_feasible(x): 
            return 0
        peso_actual = total_weight(x)
        valor_actual = total_value(x)
        cap = capacity - peso_actual
        L = []
        for i in range(len(x), n):
            if weights[i] > 0:
                ratio = values[i] / weights[i]
                L.append((ratio, weights[i], values[i]))
        L.sort(key=lambda item: item[0], reverse=True)
        
        ub = valor_actual
        for ratio, wi, vi in L:
            if wi <= cap:
                ub += vi
                cap -= wi
            else:
                break
        return ub

    #COTA CON ÍTEMS COMPLETOS EN ORDEN ORIGINAL
    @staticmethod
    def whole_items_order(x: List[int]) -> float:
        """TODO: Implementar cota con ítems completos en orden original."""
        if not is_feasible(x):
            return 0
        
        peso_actual = total_weight(x)
        valor_actual = total_value(x)
        cap = capacity - peso_actual

        ub = valor_actual
        for i in range(len(x), n):
            wi = weights[i]
            vi = values[i]
            if wi <= cap:
                ub += vi
                cap -= wi
            else:
                break
        return ub

    #COTA TRIVIAL (VALOR ACTUAL SI FACTIBLE)
    @staticmethod
    def current_value(x: List[int]) -> float:
        if not is_feasible(x):
            return 0
        return total_value(x)
    

    #COTA FLOJA (VALOR ACTUAL + SUMA DE VALORES RESTANTES)
    @staticmethod
    def remaining_value_sum(x: List[int]) -> float:
        if not is_feasible(x):
            return 0
        
        ub = total_value(x)
        for i in range(len(x), n):
            ub += values[i]
        return ub


In [4]:
# Ejecuciones para distintas funciones de cota
def test_with_bound(bound_func, name: str):
    reset_stats()
    branch_and_bound([], bound_func)
    print(f"\nBound: {name}")
    print("Mejor valor:", optimal)
    print("Selección óptima:", best_solution)
    if best_solution is not None:
        print("Pesos elegidos:", [weights[i] for i in range(n) if best_solution[i] == 1])
        print("Valores elegidos:", [values[i] for i in range(n) if best_solution[i] == 1])
    print("Nodos generados:", nodes_generated)
    print("Nodos expandidos:", nodes_expanded)
    print("Nodos podados:", nodes_pruned)

# Corre pruebas con varias cotas
test_with_bound(BoundStrategies.fractional,      "Fraccional (linear relaxation)")
test_with_bound(BoundStrategies.whole_items_ratio, "Sólo ítems completos (orden ratio)")
test_with_bound(BoundStrategies.whole_items_order, "Sólo ítems completos (orden original)")
test_with_bound(BoundStrategies.current_value,   "Sólo valor actual (trivial)")
test_with_bound(BoundStrategies.remaining_value_sum, "Suma de valores restantes (muy floja)")



Bound: Fraccional (linear relaxation)
Mejor valor: 7
Selección óptima: [1, 1, 0, 0]
Pesos elegidos: [2, 3]
Valores elegidos: [3, 4]
Nodos generados: 8
Nodos expandidos: 4
Nodos podados: 4

Bound: Sólo ítems completos (orden ratio)
Mejor valor: 7
Selección óptima: [1, 1, 0, 0]
Pesos elegidos: [2, 3]
Valores elegidos: [3, 4]
Nodos generados: 8
Nodos expandidos: 4
Nodos podados: 4

Bound: Sólo ítems completos (orden original)
Mejor valor: 7
Selección óptima: [1, 1, 0, 0]
Pesos elegidos: [2, 3]
Valores elegidos: [3, 4]
Nodos generados: 8
Nodos expandidos: 4
Nodos podados: 4

Bound: Sólo valor actual (trivial)
Mejor valor: 7
Selección óptima: [1, 1, 0, 0]
Pesos elegidos: [2, 3]
Valores elegidos: [3, 4]
Nodos generados: 8
Nodos expandidos: 4
Nodos podados: 4

Bound: Suma de valores restantes (muy floja)
Mejor valor: 7
Selección óptima: [1, 1, 0, 0]
Pesos elegidos: [2, 3]
Valores elegidos: [3, 4]
Nodos generados: 22
Nodos expandidos: 11
Nodos podados: 11


## Exploración de estrategias: BFS, DFS y Best-First (Heap)

En Branch & Bound, el orden en que se exploran los nodos puede afectar la eficiencia de la búsqueda y la rapidez con la que se encuentra la solución óptima. Las estrategias más comunes son:

- **DFS (Depth-First Search):** Se exploran los nodos hijos de cada rama antes de retroceder. Usualmente se implementa con una pila (stack).
- **BFS (Breadth-First Search):** Se exploran todos los nodos de un nivel antes de pasar al siguiente. Se implementa con una cola (queue).
- **Best-First (Heap):** Se exploran primero los nodos con mejor cota (bound), usando una cola de prioridad (heap).

A continuación puedes probar e implementar cada estrategia y comparar métricas como nodos generados, podados y expandidos.

In [ ]:
from collections import deque
import heapq

def branch_and_bound_dfs(bound_func):
    reset_stats()
    stack = [([])]
    global optimal, best_solution, nodes_generated, nodes_pruned, nodes_expanded
    while stack:
        x = stack.pop()
        # cota y factibilidad del nodo actual
        if not is_feasible(x):
            nodes_pruned += 1
            continue
        if bound_func(x) <= optimal:
            nodes_pruned += 1
            continue
        if len(x) == n:
            nodes_expanded += 1
            val = total_value(x)
            if val > optimal:
                optimal = val; best_solution = x
            continue
        # expandir hijos (primero incluir)
        children = generate_children(x)
        nodes_generated += len(children)
        stack.extend(children)

def branch_and_bound_bfs(bound_func):
    """BFS con Branch & Bound (cola FIFO)."""
    reset_stats()
    global optimal, best_solution, nodes_generated, nodes_pruned, nodes_expanded

    q = deque([[]])  # FIFO
    while q:
        x = q.popleft()

        # Poda por capacidad
        if not is_feasible(x):
            nodes_pruned += 1
            continue

        # Poda por cota
        ub = bound_func(x)
        if ub <= optimal:
            nodes_pruned += 1
            continue

        # Hoja: evaluar
        if len(x) == n:
            val = total_value(x)
            if val > optimal:
                optimal = val
                best_solution = x[:]
            continue

        nodes_expanded += 1
        children = generate_children(x)
        nodes_generated += len(children)

        q.extend(children)

def branch_and_bound_best_first(bound_func):
    """Best-First con heap priorizando mayor cota."""
    reset_stats()
    global optimal, best_solution, nodes_generated, nodes_pruned, nodes_expanded

    start = []
    start_ub = bound_func(start)
    heap = [(-start_ub, 0, start)]

    while heap:
        neg_ub, _, x = heapq.heappop(heap)
        ub = -neg_ub

        if ub <= optimal:
            nodes_pruned += 1
            continue

        # Poda por capacidad
        if not is_feasible(x):
            nodes_pruned += 1
            continue

        # Hoja: evaluar
        if len(x) == n:
            val = total_value(x)
            if val > optimal:
                optimal = val
                best_solution = x[:] 
            continue

        # Nodo interno: expandir
        nodes_expanded += 1
        children = generate_children(x)
        nodes_generated += len(children)

        # Insertar hijos con su cota como prioridad
        for child in children:
            child_ub = bound_func(child)
            if child_ub <= optimal:
                nodes_pruned += 1
                continue
            heapq.heappush(heap, (-child_ub, len(child), child))
print("DFS:")
try:
    branch_and_bound_dfs(BoundStrategies.fractional)
    print("Mejor valor:", optimal, "| Nodos generados:", nodes_generated)
except NotImplementedError as e:
    print(e)

print("\nBFS:")
try:
    branch_and_bound_bfs(BoundStrategies.fractional)
    print("Mejor valor:", optimal, "| Nodos generados:", nodes_generated)
except NotImplementedError as e:
    print(e)

print("\nBest-First (heap):")
try:
    branch_and_bound_best_first(BoundStrategies.fractional)
    print("Mejor valor:", optimal, "| Nodos generados:", nodes_generated)
except NotImplementedError as e:
    print(e)


DFS:
Mejor valor: 7 | Nodos generados: 24

BFS:
Mejor valor: 7 | Nodos generados: 24

Best-First (heap):
Mejor valor: 7 | Nodos generados: 8
